In [28]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

from plotter import *

In [29]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.2

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [30]:
year = '2018'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

ddbthr = 0.64

In [31]:
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
coffeadir_prefix = '/myeosdir/ggf-vbf/outfiles-ddb2/'

repickle = True
# Check if pickle exists, remove it if it does
picklename = year+'/cutflow.pkl'
if os.path.isfile(picklename):
    repickle = False

In [32]:
cutflow = pickle.load(open(year+'/cutflow.pkl','rb'))
cutflow = cutflow.sum('genflavor').integrate('region','muoncontrol')

In [33]:
cutflow.integrate('process','muondata').values()

{(): array([5.1097690e+07, 2.1904928e+07, 2.1050041e+07, 2.1012888e+07,
        7.0518700e+05, 7.0518700e+05, 2.5061900e+05, 8.0598000e+04,
        1.2456000e+04, 1.1162000e+04, 6.0010000e+03, 3.2000000e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00])}

In [34]:
cutflow.integrate('process','ttbar').values()

{(): array([2.22210610e+07, 2.22210610e+07, 2.22210610e+07, 2.22059247e+07,
        4.92557740e+05, 4.92557740e+05, 1.88897203e+05, 9.69234623e+04,
        1.35705931e+04, 9.41536574e+03, 5.32155996e+03, 3.66590073e+01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00])}

In [35]:
df1 = pd.DataFrame([])

df1['QCD'] = cutflow.values()[('QCD',)]
df1['Wjets'] = cutflow.values()[('Wjets',)]
df1['Zjets'] = cutflow.values()[('Zjets',)]
df1['VV'] = cutflow.values()[('VV',)]
df1['ttbar'] = cutflow.values()[('ttbar',)]
df1['singlet'] = cutflow.values()[('singlet',)]
df1['data'] = cutflow.values()[('muondata',)]

df1 = df1.astype('int')[:12]
df1.index=['nothing','muontrigger','lumimask','metfilter','minjetkinmu', 'jetid', 'n2ddt', 'ak4btagMedium08', 'onemuon', 'muonkin', 'muonDphiAK8','ddbpass']
df1[1:12].to_latex(buf=year+'/cutflow-muoncr.tex')

In [36]:
df1

,QCD,Wjets,Zjets,VV,ttbar,singlet,data
nothing,1542533361,25156826,3340706,2372304,22221060,3412929,51097690
muontrigger,1542533361,25156826,3340706,2372304,22221060,3412929,21904928
lumimask,1542533361,25156826,3340706,2372304,22221060,3412929,21050041
metfilter,1540122249,25125157,3333096,2371047,22205924,3410935,21012888
minjetkinmu,56594272,851948,348268,11054,492557,42035,705187
jetid,56594272,851948,348268,11054,492557,42035,705187
n2ddt,15345526,352917,160380,6584,188897,21583,250619
ak4btagMedium08,1431554,31053,21034,708,96923,11879,80598
onemuon,5537,1620,326,35,13570,1362,12456
muonkin,1064,1031,170,20,9415,814,11162
